# 0. 패키지 불러오기 및 데이터 확인

In [1]:
!pip install catboost
!pip install lightgbm

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import math

from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import ExtraTreesClassifier, ExtraTreesRegressor, RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier, GradientBoostingRegressor, HistGradientBoostingClassifier, HistGradientBoostingRegressor
from sklearn.model_selection import StratifiedKFold, train_test_split, KFold # KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, roc_auc_score, f1_score, confusion_matrix 
# make_scorer : MSE 대신 사용자가 정의한 손실함수를 사용하고 싶을 때
from sklearn.preprocessing import LabelEncoder, StandardScaler
# from sklearn.svm import * # SVC, SVR
# from sklearn.inspection import *
# from sklearn.linear_model import * # LogisticRegression, RANSACRegressor, Ridge, Lasso, ElasticNet
# from sklearn.decomposition import * # PCA

# from category_encoders.ordinal import OrdinalEncoder
from catboost import CatBoostClassifier, CatBoostRegressor
# from sklearn.cluster import KMeans
# from kmodes.kmodes import KModes
import optuna
from optuna import Trial
from optuna.samplers import TPESampler
# from pycaret.classification import *
import random
from pathlib import Path

from lightgbm import LGBMClassifier, LGBMRegressor
import lightgbm as lgb
# import xgboost as xgb
from datetime import datetime
import calendar
from collections import Counter

import warnings
warnings.filterwarnings(action = 'ignore') # 경고메세지를 무시하거나 숨긴다.

In [6]:
path = '/home/studio-lab-user/MYDATA/Construction Machine Oil/open/'

Rdata_train = pd.read_csv(path + 'train.csv')
Rdata_test = pd.read_csv(path + 'test.csv')

print(Rdata_train.shape) # (14095, 54) -> 생각보다 데이터 수가 적다.
print(Rdata_train.size)
print(Rdata_test.shape) # (6041, 19)
print(Rdata_test.size)
Rdata_train.head()

(14095, 54)
761130
(6041, 19)
114779


,ID,COMPONENT_ARBITRARY,ANONYMOUS_1,YEAR,SAMPLE_TRANSFER_DAY,ANONYMOUS_2,AG,AL,B,BA,...,U25,U20,U14,U6,U4,V,V100,V40,ZN,Y_LABEL
0,TRAIN_00000,COMPONENT3,1486,2011,7,200,0,3,93,0,...,NaN,NaN,NaN,NaN,NaN,0,NaN,154.0,75,0
1,TRAIN_00001,COMPONENT2,1350,2021,51,375,0,2,19,0,...,2.0,4.0,6.0,216.0,1454.0,0,NaN,44.0,652,0
2,TRAIN_00002,COMPONENT2,2415,2015,2,200,0,110,1,1,...,0.0,3.0,39.0,11261.0,41081.0,0,NaN,72.6,412,1
3,TRAIN_00003,COMPONENT3,7389,2010,2,200,0,8,3,0,...,NaN,NaN,NaN,NaN,NaN,0,NaN,133.3,7,0
4,TRAIN_00004,COMPONENT3,3954,2015,4,200,0,1,157,0,...,NaN,NaN,NaN,NaN,NaN,0,NaN,133.1,128,0


# 1. CatBoost

new_var_set = ['FOPTIMETHGLY', 'FE', 'ZN', 'NI', 'BA', 
               'FNOX', 'CR', 'SI', 'CA', 'U75', 
               'K', 'SN', 'AL', 'V40', 'NA', 
               'FH2O', 'U4', 'ANONYMOUS_1', 'S', 'SB', 
               'FTBN', 'CU', 'B', 'PQINDEX', 'ANONYMOUS_2',
               'COMPONENT_ARBITRARY', 'YEAR']  
주 변수 -> 'COMPONENT_ARBITRARY', 'ANONYMOUS_1', 'YEAR', 'ANONYMOUS_2', 'CR', 'CU', 'FE', 'NI', 'PQINDEX', 'V40', 'ZN' -> 11개  
부 변수(결측 없음) -> 'BA', 'SI', 'CA', 'SN', 'AL', 'NA', 'S', 'SB', 'B' -> 9개  
부 별수(결측 있음) -> 'FH2O', 'FNOX', 'FOPTIMETHGLY', 'FTBN', 'K', 'U75', 'U4' -> 7개  

In [76]:
train1 = Rdata_train.copy()
test1 = Rdata_test.copy()

In [77]:
# AL과 BA만 추가하여 MODEL1을 만들자.
train2 = train1.loc[:, ['COMPONENT_ARBITRARY', 'ANONYMOUS_1', 'YEAR', 'ANONYMOUS_2', 'AG',
                                        'CO', 'CR', 'CU', 'FE', 'H2O', 'MN', 'MO', 'NI', 'PQINDEX', 'TI', 'V',
                                        'V40', 'ZN', 'Y_LABEL', 'AL', 'BA']]
test2 = test1.drop(['ID'], axis = 1)

print(train2.shape) # 변수 20개
print(test2.shape) # 변수 18개

(14095, 21)
(6041, 18)


In [78]:
# 이 작업을 수행하면 범주형 변수인 COMPONENT_ARBITRARY와 YEAR를 숫자로 바꿔준다.
# 단 OneHotEncoder가 아님, 내가 쓴건 LabelEncoder
# OneHotEncoder -> 개 : 1 0 0, 고양이 : 0 1 0, 사자 : 0 0 1
# LabelEncoder -> 개 : 0, 고양이 : 1, 사자 : 2

le1 = LabelEncoder()
le2 = LabelEncoder()

train2['COMPONENT_ARBITRARY_category'] = le1.fit_transform(train2['COMPONENT_ARBITRARY'])
train2['YEAR_category'] = le2.fit_transform(train2['YEAR'])

test2['COMPONENT_ARBITRARY_category'] = le1.transform(test2['COMPONENT_ARBITRARY'])
test2['YEAR_category'] = le2.transform(test2['YEAR'])

# 원래 범주형 변수는 제거해준다.
train3 = train2.drop(['COMPONENT_ARBITRARY', 'YEAR'], axis = 1)
test3 = test2.drop(['COMPONENT_ARBITRARY', 'YEAR'], axis = 1)

In [79]:
# 범주형 변수와 숫자형 변수를 나눠준다.
categorical_features = ['COMPONENT_ARBITRARY_category', 'YEAR_category']
# numeric_features_train = ['ANONYMOUS_1', 'ANONYMOUS_2', 'AG', 'CO', 'CR', 'CU', 
#                     'FE', 'H2O', 'MN', 'MO', 'NI', 'PQINDEX', 'TI', 'V', 'V40', 'ZN', 'AL', 'BA']
# numeric_features_test = ['ANONYMOUS_1', 'ANONYMOUS_2', 'AG', 'CO', 'CR', 'CU', 
#                     'FE', 'H2O', 'MN', 'MO', 'NI', 'PQINDEX', 'TI', 'V', 'V40', 'ZN']

print(train3.shape)
print(test3.shape)

(14095, 21)
(6041, 18)


In [80]:
X_train = train3.drop(['Y_LABEL'], axis = 1)
y_train = train3['Y_LABEL']
X_test = test3

In [81]:
# 교차 검증을 위해 train데이터를 partrain과 val로 나눠준다.

X_partrain, X_val, y_partrain, y_val = train_test_split(X_train, y_train, test_size = 0.3, random_state = 39, stratify = y_train)
print(X_partrain.shape)
print(X_val.shape)
print(y_partrain.shape)
print(y_val.shape)

(9866, 20)
(4229, 20)
(9866,)
(4229,)


In [82]:
# 교차검증 패키지인 optuna 사용을 한다.
# 이 작업은 어떤 초모수일때, 모델이 가장 좋은지 적합해준다.

def objective(trial : Trial) -> float :

    params_cat = {
        "random_state" : 39,
        'learning_rate' : trial.suggest_loguniform('learning_rate', 0.001, 1), # 학습률이 0.005와 0.5 사이의 랜덤한 값으로 정해줌
        "n_estimators" : trial.suggest_int("n_estimators", 100, 1000), # 400과 1000 사이에 랜덤한 값으로 정해줌
        "max_depth" : trial.suggest_int("max_depth", 3, 16) # 3과 10 사이에 랜덤한 값으로 정해줌
  }
    
    model = CatBoostClassifier(**params_cat) # Catboost가 AutoML에서 가장 좋아서 Catboost로 했다.
    model.fit(X_partrain, y_partrain, eval_set = [(X_val, y_val)],
              early_stopping_rounds = 100, cat_features = categorical_features, verbose = False)

    cat_pred = model.predict(X_val)
    AUC = roc_auc_score(y_val, cat_pred)
    
    return AUC

In [15]:
# 이 작업을 해주면 어떤 초모수가 가장 좋은지를 얘가 판단을 해준다.
# 나는 총 100번 돌렸지만, 원래는 1000번 정도 돌리는게 좋다.

sampler = TPESampler(seed = 39)
study = optuna.create_study(
    study_name = "cat_parameter_opt",
    direction = "maximize",
    sampler = sampler)
study.optimize(objective, n_trials = 100)

[I 2022-12-10 12:00:05,513] A new study created in memory with name: cat_parameter_opt
[W 2022-12-10 12:00:10,014] Trial 0 failed because of the following error: KeyboardInterrupt('')
Traceback (most recent call last):
  File "/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_174/1687905101.py", line 14, in objective
    model.fit(X_partrain, y_partrain, eval_set = [(X_val, y_val)],
  File "/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/catboost/core.py", line 5128, in fit
    self._fit(X, y, cat_features, text_features, embedding_features, None, sample_weight, None, None, None, None, baseline, use_best_model,
  File "/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/catboost/core.py", line 2355, in _fit
    self._train(
  File "/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/catboost/core.py", 

KeyboardInterrupt: 

In [ ]:
# 가장 좋은 초모수를 알려준다. 

print("Best Score :", study.best_value)
print("Best trial :", study.best_trial.params)

Best Score : 0.7665005428446204
Best trial : {'learning_rate': 0.03142344166841527, 'n_estimators': 513, 'max_depth': 6}


In [83]:
# 교차 검증해서 얻은 초모수를 가지고 train 데이터의 예측 불량률을 파악한다.

n_fold = 5
cv = StratifiedKFold(n_splits = n_fold, shuffle = True, random_state = 39)

cat_val = np.zeros((X_train.shape[0], 2))
cat_partrain = np.zeros((X_partrain.shape[0], 2))

print(cat_val.shape)
print(cat_partrain.shape)

for i, (i_trn, i_val) in enumerate(cv.split(X_train, y_train), 1):
    print(f'training model for CV #{i}')
    optuna_cat = CatBoostClassifier(
        random_state = 39,
        learning_rate = 0.03142344166841527, 
        n_estimators = 513, 
        max_depth = 6)

    optuna_cat.fit(X_train.loc[i_trn, :], y_train[i_trn], verbose = False, 
                   cat_features = categorical_features, early_stopping_rounds = 100)

    cat_val[i_val, :] = optuna_cat.predict_proba(X_train.loc[i_val, :])
    cat_partrain += optuna_cat.predict_proba(X_partrain) / n_fold

(14095, 2)
(9866, 2)
training model for CV #1
training model for CV #2
training model for CV #3
training model for CV #4
training model for CV #5


In [85]:
# 정상률 말고 불량률만 train 데이터에 따로 붙여서 train 데이터의 변수는 총 22개로 늘어났다.

print(cat_val.shape)
print(cat_partrain.shape)

train_teacher = train3.copy()

train_teacher['Cat_prob1'] = cat_val[:, 1]
print(train_teacher.shape)

(14095, 2)
(9866, 2)
(14095, 22)


# 2. LightGBM

In [89]:
train1 = Rdata_train.copy()
test1 = Rdata_test.copy()

train2 = train1.loc[:, ['COMPONENT_ARBITRARY', 'ANONYMOUS_1', 'YEAR', 'ANONYMOUS_2', 'AG',
                                        'CO', 'CR', 'CU', 'FE', 'H2O', 'MN', 'MO', 'NI', 'PQINDEX', 'TI', 'V',
                                        'V40', 'ZN', 'Y_LABEL', 'AL', 'BA']]
test2 = test1.drop(['ID'], axis = 1)

le1 = LabelEncoder()
le2 = LabelEncoder()

train2['COMPONENT_ARBITRARY_category'] = le1.fit_transform(train2['COMPONENT_ARBITRARY'])
train2['YEAR_category'] = le2.fit_transform(train2['YEAR'])

test2['COMPONENT_ARBITRARY_category'] = le1.transform(test2['COMPONENT_ARBITRARY'])
test2['YEAR_category'] = le2.transform(test2['YEAR'])

train3 = train2.drop(['COMPONENT_ARBITRARY', 'YEAR'], axis = 1)
test3 = test2.drop(['COMPONENT_ARBITRARY', 'YEAR'], axis = 1)

categorical_features = ['COMPONENT_ARBITRARY_category', 'YEAR_category']

X_train = train3.drop(['Y_LABEL'], axis = 1)
y_train = train3['Y_LABEL']
X_test = test3

X_partrain, X_val, y_partrain, y_val = train_test_split(X_train, y_train, test_size = 0.3, random_state = 39, stratify = y_train)
print(X_partrain.shape)
print(X_val.shape)
print(y_partrain.shape)
print(y_val.shape)

(9866, 20)
(4229, 20)
(9866,)
(4229,)


In [90]:
def objective(trial : Trial) -> float :

    params_lgb = {
        "random_state" : 39,
        'learning_rate' : trial.suggest_loguniform('learning_rate', 0.001, 1),
        "n_estimators" : trial.suggest_int("n_estimators", 100, 1000), 
        "max_depth" : trial.suggest_int("max_depth", 3, 16),
        'reg_alpha' : trial.suggest_loguniform('reg_alpha', 0.001, 1),
        'reg_lambda' : trial.suggest_loguniform('reg_lambda', 0.001, 1)
        
    }
    
    model = lgb.LGBMClassifier(**params_lgb)
    model.fit(X_partrain, y_partrain, eval_set = [(X_val, y_val)],
              early_stopping_rounds = 100, verbose = False)

    lgb_pred = model.predict(X_val)
    AUC = roc_auc_score(y_val, lgb_pred)
    
    return AUC

In [33]:
sampler = TPESampler(seed = 39)
study = optuna.create_study(
    study_name = "lgb_parameter_opt",
    direction = "maximize",
    sampler = sampler)
study.optimize(objective, n_trials = 100)

[I 2022-12-10 13:52:21,822] A new study created in memory with name: lgb_parameter_opt
[I 2022-12-10 13:52:22,263] Trial 0 finished with value: 0.7652447670637978 and parameters: {'learning_rate': 0.04371872304807245, 'n_estimators': 818, 'max_depth': 14, 'reg_alpha': 0.002323537042351288, 'reg_lambda': 0.0639743703819749}. Best is trial 0 with value: 0.7652447670637978.
[I 2022-12-10 13:52:22,656] Trial 1 finished with value: 0.7641182570533993 and parameters: {'learning_rate': 0.03771757391455586, 'n_estimators': 517, 'max_depth': 9, 'reg_alpha': 0.07909308741230786, 'reg_lambda': 0.5984000779343428}. Best is trial 0 with value: 0.7652447670637978.
[I 2022-12-10 13:52:22,882] Trial 2 finished with value: 0.7658542139925004 and parameters: {'learning_rate': 0.27958170383769165, 'n_estimators': 950, 'max_depth': 15, 'reg_alpha': 0.017558701225637952, 'reg_lambda': 0.30021741896396037}. Best is trial 2 with value: 0.7658542139925004.
[I 2022-12-10 13:52:23,057] Trial 3 finished with val

In [34]:
print("Best Score :", study.best_value)
print("Best trial :", study.best_trial.params)

Best Score : 0.7762516937038618
Best trial : {'learning_rate': 0.988106545136767, 'n_estimators': 809, 'max_depth': 8, 'reg_alpha': 0.0010213691245675122, 'reg_lambda': 0.04676899086488728}


In [91]:
# 교차 검증해서 얻은 초모수를 가지고 train 데이터의 예측 불량률을 파악한다.

n_fold = 5
cv = StratifiedKFold(n_splits = n_fold, shuffle = True, random_state = 39)

lgb_val = np.zeros((X_train.shape[0], 2))
lgb_partrain = np.zeros((X_partrain.shape[0], 2))

print(lgb_val.shape)
print(lgb_partrain.shape)

for i, (i_trn, i_val) in enumerate(cv.split(X_train, y_train), 1):
    print(f'training model for CV #{i}')
    optuna_lgb = lgb.LGBMClassifier(
        random_state = 39,
        learning_rate = 0.988106545136767, 
        n_estimators = 809, 
        max_depth = 8,
        reg_alpha = 0.0010213691245675122,
        reg_lambda = 0.04676899086488728)

    optuna_lgb.fit(X_train.loc[i_trn, :], y_train[i_trn], verbose = False)

    lgb_val[i_val, :] = optuna_lgb.predict_proba(X_train.loc[i_val, :])
    lgb_partrain += optuna_lgb.predict_proba(X_partrain) / n_fold

(14095, 2)
(9866, 2)
training model for CV #1
training model for CV #2
training model for CV #3
training model for CV #4
training model for CV #5


In [92]:
print(lgb_val.shape)

train_teacher['Lgb_prob1'] = lgb_val[:, 1]
print(train_teacher.shape)

(14095, 2)
(14095, 23)


# 3. RandomForest

In [93]:
train1 = Rdata_train.copy()
test1 = Rdata_test.copy()

train2 = train1.loc[:, ['COMPONENT_ARBITRARY', 'ANONYMOUS_1', 'YEAR', 'ANONYMOUS_2', 'AG',
                                        'CO', 'CR', 'CU', 'FE', 'H2O', 'MN', 'MO', 'NI', 'PQINDEX', 'TI', 'V',
                                        'V40', 'ZN', 'Y_LABEL', 'AL', 'BA']]
test2 = test1.drop(['ID'], axis = 1)

le1 = LabelEncoder()
le2 = LabelEncoder()

train2['COMPONENT_ARBITRARY_category'] = le1.fit_transform(train2['COMPONENT_ARBITRARY'])
train2['YEAR_category'] = le2.fit_transform(train2['YEAR'])

test2['COMPONENT_ARBITRARY_category'] = le1.transform(test2['COMPONENT_ARBITRARY'])
test2['YEAR_category'] = le2.transform(test2['YEAR'])

train3 = train2.drop(['COMPONENT_ARBITRARY', 'YEAR'], axis = 1)
test3 = test2.drop(['COMPONENT_ARBITRARY', 'YEAR'], axis = 1)

categorical_features = ['COMPONENT_ARBITRARY_category', 'YEAR_category']

X_train = train3.drop(['Y_LABEL'], axis = 1)
y_train = train3['Y_LABEL']
X_test = test3

X_partrain, X_val, y_partrain, y_val = train_test_split(X_train, y_train, test_size = 0.3, random_state = 39, stratify = y_train)
print(X_partrain.shape)
print(X_val.shape)
print(y_partrain.shape)
print(y_val.shape)

(9866, 20)
(4229, 20)
(9866,)
(4229,)


In [94]:
def objective(trial : Trial) -> float :

    params_rf = {
        "random_state" : 39,
        "n_estimators" : trial.suggest_int("n_estimators", 100, 1000), 
        "max_depth" : trial.suggest_int("max_depth", 3, 16) 
        }
    
    model = RandomForestClassifier(**params_rf)
    model.fit(X_partrain, y_partrain)

    rf_pred = model.predict(X_val)
    AUC = roc_auc_score(y_val, rf_pred)
    
    return AUC

In [5]:
sampler = TPESampler(seed = 39)
study = optuna.create_study(
    study_name = "rf_parameter_opt",
    direction = "maximize",
    sampler = sampler)
study.optimize(objective, n_trials = 100)

[I 2022-12-10 14:23:52,330] A new study created in memory with name: rf_parameter_opt
[I 2022-12-10 14:24:01,192] Trial 0 finished with value: 0.7612189081804822 and parameters: {'n_estimators': 592, 'max_depth': 14}. Best is trial 0 with value: 0.7612189081804822.
[I 2022-12-10 14:24:06,222] Trial 1 finished with value: 0.7477562899792889 and parameters: {'n_estimators': 839, 'max_depth': 4}. Best is trial 0 with value: 0.7612189081804822.
[I 2022-12-10 14:24:13,372] Trial 2 finished with value: 0.758448825077989 and parameters: {'n_estimators': 642, 'max_depth': 10}. Best is trial 0 with value: 0.7612189081804822.
[I 2022-12-10 14:24:18,662] Trial 3 finished with value: 0.7570637835267428 and parameters: {'n_estimators': 517, 'max_depth': 9}. Best is trial 0 with value: 0.7612189081804822.
[I 2022-12-10 14:24:27,845] Trial 4 finished with value: 0.7612189081804822 and parameters: {'n_estimators': 670, 'max_depth': 15}. Best is trial 0 with value: 0.7612189081804822.
[I 2022-12-10 14:

In [6]:
print("Best Score :", study.best_value)
print("Best trial :", study.best_trial.params)

Best Score : 0.7639889912829753
Best trial : {'n_estimators': 121, 'max_depth': 16}


In [95]:
n_fold = 5
cv = StratifiedKFold(n_splits = n_fold, shuffle = True, random_state = 39)

rf_val = np.zeros((X_train.shape[0], 2))
rf_partrain = np.zeros((X_partrain.shape[0], 2))

print(rf_val.shape)
print(rf_partrain.shape)

for i, (i_trn, i_val) in enumerate(cv.split(X_train, y_train), 1):
    print(f'training model for CV #{i}')
    optuna_rf = RandomForestClassifier(
        random_state = 39,
        n_estimators = 121, 
        max_depth = 16)

    optuna_rf.fit(X_train.loc[i_trn, :], y_train[i_trn])

    rf_val[i_val, :] = optuna_rf.predict_proba(X_train.loc[i_val, :])
    rf_partrain += optuna_rf.predict_proba(X_partrain) / n_fold

(14095, 2)
(9866, 2)
training model for CV #1
training model for CV #2
training model for CV #3
training model for CV #4
training model for CV #5


In [96]:
print(rf_val.shape)

train_teacher['Rf_prob1'] = rf_val[:, 1]
print(train_teacher.shape)

(14095, 2)
(14095, 24)


# 4. GBM

In [97]:
train1 = Rdata_train.copy()
test1 = Rdata_test.copy()

train2 = train1.loc[:, ['COMPONENT_ARBITRARY', 'ANONYMOUS_1', 'YEAR', 'ANONYMOUS_2', 'AG',
                                        'CO', 'CR', 'CU', 'FE', 'H2O', 'MN', 'MO', 'NI', 'PQINDEX', 'TI', 'V',
                                        'V40', 'ZN', 'Y_LABEL', 'AL', 'BA']]
test2 = test1.drop(['ID'], axis = 1)

le1 = LabelEncoder()
le2 = LabelEncoder()

train2['COMPONENT_ARBITRARY_category'] = le1.fit_transform(train2['COMPONENT_ARBITRARY'])
train2['YEAR_category'] = le2.fit_transform(train2['YEAR'])

test2['COMPONENT_ARBITRARY_category'] = le1.transform(test2['COMPONENT_ARBITRARY'])
test2['YEAR_category'] = le2.transform(test2['YEAR'])

train3 = train2.drop(['COMPONENT_ARBITRARY', 'YEAR'], axis = 1)
test3 = test2.drop(['COMPONENT_ARBITRARY', 'YEAR'], axis = 1)

categorical_features = ['COMPONENT_ARBITRARY_category', 'YEAR_category']

X_train = train3.drop(['Y_LABEL'], axis = 1)
y_train = train3['Y_LABEL']
X_test = test3

X_partrain, X_val, y_partrain, y_val = train_test_split(X_train, y_train, test_size = 0.3, random_state = 39, stratify = y_train)
print(X_partrain.shape)
print(X_val.shape)
print(y_partrain.shape)
print(y_val.shape)

(9866, 20)
(4229, 20)
(9866,)
(4229,)


In [98]:
def objective(trial : Trial) -> float :

    params_gbm = {
        "random_state" : 39,
        "n_estimators" : trial.suggest_int("n_estimators", 100, 1000), 
        "max_depth" : trial.suggest_int("max_depth", 3, 16),
    }
    
    model = GradientBoostingClassifier(**params_gbm)
    model.fit(X_partrain, y_partrain)

    gbm_pred = model.predict(X_val)
    AUC = roc_auc_score(y_val, gbm_pred)
    
    return AUC

In [64]:
sampler = TPESampler(seed = 39)
study = optuna.create_study(
    study_name = "gbm_parameter_opt",
    direction = "maximize",
    sampler = sampler)
study.optimize(objective, n_trials = 100)

[I 2022-12-10 15:51:47,016] A new study created in memory with name: gbm_parameter_opt
[I 2022-12-10 15:52:57,465] Trial 0 finished with value: 0.7628255993491594 and parameters: {'n_estimators': 592, 'max_depth': 14}. Best is trial 0 with value: 0.7628255993491594.
[I 2022-12-10 15:53:17,946] Trial 1 finished with value: 0.7679410863194561 and parameters: {'n_estimators': 839, 'max_depth': 4}. Best is trial 1 with value: 0.7679410863194561.
[I 2022-12-10 15:54:01,897] Trial 2 finished with value: 0.7661127455333484 and parameters: {'n_estimators': 642, 'max_depth': 10}. Best is trial 1 with value: 0.7679410863194561.
[I 2022-12-10 15:54:32,567] Trial 3 finished with value: 0.7619576208796088 and parameters: {'n_estimators': 517, 'max_depth': 9}. Best is trial 1 with value: 0.7679410863194561.
[I 2022-12-10 15:56:00,730] Trial 4 finished with value: 0.7608311108692102 and parameters: {'n_estimators': 670, 'max_depth': 15}. Best is trial 1 with value: 0.7679410863194561.
[I 2022-12-10 1

In [65]:
print("Best Score :", study.best_value)
print("Best trial :", study.best_trial.params)

Best Score : 0.7713943801974866
Best trial : {'n_estimators': 937, 'max_depth': 7}


In [99]:
n_fold = 5
cv = StratifiedKFold(n_splits = n_fold, shuffle = True, random_state = 39)

gbm_val = np.zeros((X_train.shape[0], 2))
gbm_partrain = np.zeros((X_partrain.shape[0], 2))

print(gbm_val.shape)
print(gbm_partrain.shape)

for i, (i_trn, i_val) in enumerate(cv.split(X_train, y_train), 1):
    print(f'training model for CV #{i}')
    optuna_gbm = GradientBoostingClassifier(
        random_state = 39,
        n_estimators = 937, 
        max_depth = 7)

    optuna_gbm.fit(X_train.loc[i_trn, :], y_train[i_trn])

    gbm_val[i_val, :] = optuna_gbm.predict_proba(X_train.loc[i_val, :])
    gbm_partrain += optuna_gbm.predict_proba(X_partrain) / n_fold

(14095, 2)
(9866, 2)
training model for CV #1
training model for CV #2
training model for CV #3
training model for CV #4
training model for CV #5


In [100]:
print(gbm_val.shape)

train_teacher['Gbm_prob1'] = gbm_val[:, 1]
print(train_teacher.shape)

(14095, 2)
(14095, 25)


# 5. HGBM

In [101]:
train1 = Rdata_train.copy()
test1 = Rdata_test.copy()

In [102]:
train2 = train1.loc[:, ['COMPONENT_ARBITRARY', 'ANONYMOUS_1', 'YEAR', 'ANONYMOUS_2', 'AG',
                                        'CO', 'CR', 'CU', 'FE', 'H2O', 'MN', 'MO', 'NI', 'PQINDEX', 'TI', 'V',
                                        'V40', 'ZN', 'Y_LABEL', 'AL', 'BA']]
test2 = test1.drop(['ID'], axis = 1)

print(train2.shape) # 변수 20개
print(test2.shape) # 변수 18개

(14095, 21)
(6041, 18)


In [103]:
le1 = LabelEncoder()
le2 = LabelEncoder()

train2['COMPONENT_ARBITRARY_category'] = le1.fit_transform(train2['COMPONENT_ARBITRARY'])
train2['YEAR_category'] = le2.fit_transform(train2['YEAR'])

test2['COMPONENT_ARBITRARY_category'] = le1.transform(test2['COMPONENT_ARBITRARY'])
test2['YEAR_category'] = le2.transform(test2['YEAR'])

# 원래 범주형 변수는 제거해준다.
train3 = train2.drop(['COMPONENT_ARBITRARY', 'YEAR'], axis = 1)
test3 = test2.drop(['COMPONENT_ARBITRARY', 'YEAR'], axis = 1)

In [104]:
categorical_features = ['COMPONENT_ARBITRARY_category', 'YEAR_category']

print(train3.shape)
print(test3.shape)

(14095, 21)
(6041, 18)


In [105]:
X_train = train3.drop(['Y_LABEL'], axis = 1)
y_train = train3['Y_LABEL']
X_test = test3

In [106]:
X_partrain, X_val, y_partrain, y_val = train_test_split(X_train, y_train, test_size = 0.3, random_state = 39, stratify = y_train)
print(X_partrain.shape)
print(X_val.shape)
print(y_partrain.shape)
print(y_val.shape)

(9866, 20)
(4229, 20)
(9866,)
(4229,)


In [107]:
def objective(trial : Trial) -> float :

    params_hgb = {
        "random_state" : 39,
        'learning_rate' : trial.suggest_loguniform('learning_rate', 0.001, 1), # 학습률이 0.005와 0.5 사이의 랜덤한 값으로 정해줌
        "max_iter" : trial.suggest_int("max_iter", 100, 1000), # 400과 1000 사이에 랜덤한 값으로 정해줌
        "max_depth" : trial.suggest_int("max_depth", 3, 16), # 3과 10 사이에 랜덤한 값으로 정해줌
        'l2_regularization' : trial.suggest_loguniform('reg_lambda', 0.001, 1)
    }
    
    model = HistGradientBoostingClassifier(**params_hgb) # Catboost가 AutoML에서 가장 좋아서 Catboost로 했다.
    model.fit(X_partrain, y_partrain)

    hgb_pred = model.predict(X_val)
    AUC = roc_auc_score(y_val, hgb_pred)
    
    return AUC

In [16]:
sampler = TPESampler(seed = 39)
study = optuna.create_study(
    study_name = "hgb_parameter_opt",
    direction = "maximize",
    sampler = sampler)
study.optimize(objective, n_trials = 100)

[I 2022-12-11 01:55:37,784] A new study created in memory with name: hgb_parameter_opt
[I 2022-12-11 01:55:42,339] Trial 0 finished with value: 0.7620868866500329 and parameters: {'learning_rate': 0.04371872304807245, 'max_iter': 818, 'max_depth': 14, 'reg_lambda': 0.002323537042351288}. Best is trial 0 with value: 0.7620868866500329.
[I 2022-12-11 01:55:45,206] Trial 1 finished with value: 0.7651155012933739 and parameters: {'learning_rate': 0.0639743703819749, 'max_iter': 573, 'max_depth': 9, 'reg_lambda': 0.02596194546249675}. Best is trial 1 with value: 0.7651155012933739.
[I 2022-12-11 01:55:50,263] Trial 2 finished with value: 0.7645984382116778 and parameters: {'learning_rate': 0.07909308741230786, 'max_iter': 934, 'max_depth': 14, 'reg_lambda': 0.6812793998007773}. Best is trial 1 with value: 0.7651155012933739.
[I 2022-12-11 01:55:52,877] Trial 3 finished with value: 0.7665560447682097 and parameters: {'learning_rate': 0.5737932383737886, 'max_iter': 473, 'max_depth': 14, 'reg

In [17]:
print("Best Score :", study.best_value)
print("Best trial :", study.best_trial.params)

Best Score : 0.77000933864624
Best trial : {'learning_rate': 0.25627679018516575, 'max_iter': 103, 'max_depth': 11, 'reg_lambda': 0.33569768026131186}


In [108]:
n_fold = 5
cv = StratifiedKFold(n_splits = n_fold, shuffle = True, random_state = 39)

hgb_val = np.zeros((X_train.shape[0], 2))
hgb_partrain = np.zeros((X_partrain.shape[0], 2))

print(hgb_val.shape)
print(hgb_partrain.shape)

for i, (i_trn, i_val) in enumerate(cv.split(X_train, y_train), 1):
    print(f'training model for CV #{i}')
    optuna_hgb = HistGradientBoostingClassifier(
        random_state = 39,
        learning_rate = 0.25627679018516575, 
        max_iter = 103,
        l2_regularization = 0.33569768026131186)

    optuna_hgb.fit(X_train.loc[i_trn, :], y_train[i_trn])

    hgb_val[i_val, :] = optuna_hgb.predict_proba(X_train.loc[i_val, :])
    hgb_partrain += optuna_hgb.predict_proba(X_partrain) / n_fold

(14095, 2)
(9866, 2)
training model for CV #1
training model for CV #2
training model for CV #3
training model for CV #4
training model for CV #5


In [109]:
print(hgb_val.shape)

train_teacher['Hgb_prob1'] = hgb_val[:, 1]
print(train_teacher.shape)

(14095, 2)
(14095, 26)


########################################################################################  
########################################################################################  
# 6. Student Model
########################################################################################  
########################################################################################  

In [117]:
train_teacher.columns

Index(['ANONYMOUS_1', 'ANONYMOUS_2', 'AG', 'CO', 'CR', 'CU', 'FE', 'H2O', 'MN',
       'MO', 'NI', 'PQINDEX', 'TI', 'V', 'V40', 'ZN', 'Y_LABEL', 'AL', 'BA',
       'COMPONENT_ARBITRARY_category', 'YEAR_category', 'Cat_prob1',
       'Lgb_prob1', 'Rf_prob1', 'Gbm_prob1', 'Hgb_prob1'],
      dtype='object')

In [220]:
train3['model1_prob'] = (train_teacher['Cat_prob1'] + train_teacher['Rf_prob1']) / 2
print(train3.shape)
print(train3.columns)

(14095, 22)
Index(['ANONYMOUS_1', 'ANONYMOUS_2', 'AG', 'CO', 'CR', 'CU', 'FE', 'H2O', 'MN',
       'MO', 'NI', 'PQINDEX', 'TI', 'V', 'V40', 'ZN', 'Y_LABEL', 'AL', 'BA',
       'COMPONENT_ARBITRARY_category', 'YEAR_category', 'model1_prob'],
      dtype='object')


In [221]:
X_train2 = train3.drop(['AL', 'BA', 'Y_LABEL', 'model1_prob'], axis = 1)
y_train2 = train3['model1_prob']
print(X_train2.shape)
print(y_train2.shape)

(14095, 18)
(14095,)


In [222]:
X_partrain, X_val, y_partrain, y_val = train_test_split(X_train2, y_train2, test_size = 0.3, random_state = 39)
print(X_partrain.shape)
print(X_val.shape)
print(y_partrain.shape)
print(y_val.shape)

(9866, 18)
(4229, 18)
(9866,)
(4229,)


In [223]:
categorical_features

['COMPONENT_ARBITRARY_category', 'YEAR_category']

In [224]:
from catboost.core import CatBoostRegressor
def objective(trial : Trial) -> float :

    params_cat = {
        "random_state" : 39,
        'learning_rate' : trial.suggest_loguniform('learning_rate', 0.001, 1),
        "n_estimators" : trial.suggest_int("n_estimators", 100, 1000),
        "max_depth" : trial.suggest_int("max_depth", 3, 10)
  }
    
    model = CatBoostRegressor(**params_cat)
    model.fit(X_partrain, y_partrain, eval_set = [(X_val, y_val)],
              early_stopping_rounds = 100, cat_features = categorical_features, verbose = False)

    cat_pred = model.predict(X_val)
    MAE = mean_absolute_error(y_val, cat_pred)
    
    return MAE

In [203]:
sampler = TPESampler(seed = 39)
study = optuna.create_study(
    study_name = "cat_parameter_opt",
    direction = "minimize",
    sampler = sampler)
study.optimize(objective, n_trials = 100)

[I 2022-12-11 08:09:01,820] A new study created in memory with name: cat_parameter_opt
[I 2022-12-11 08:09:09,112] Trial 0 finished with value: 0.0795696415207403 and parameters: {'learning_rate': 0.04371872304807245, 'n_estimators': 818, 'max_depth': 9}. Best is trial 0 with value: 0.0795696415207403.
[I 2022-12-11 08:09:13,711] Trial 1 finished with value: 0.08248199234676559 and parameters: {'learning_rate': 0.002323537042351288, 'n_estimators': 642, 'max_depth': 7}. Best is trial 0 with value: 0.0795696415207403.
[I 2022-12-11 08:09:18,982] Trial 2 finished with value: 0.07993565858450426 and parameters: {'learning_rate': 0.024644795423723085, 'n_estimators': 524, 'max_depth': 8}. Best is trial 0 with value: 0.0795696415207403.
[I 2022-12-11 08:09:21,521] Trial 3 finished with value: 0.08344865191672474 and parameters: {'learning_rate': 0.5984000779343428, 'n_estimators': 834, 'max_depth': 10}. Best is trial 0 with value: 0.0795696415207403.
[I 2022-12-11 08:09:23,667] Trial 4 fini

In [204]:
print("Best Score :", study.best_value)
print("Best trial :", study.best_trial.params) 

# 최고 : 'learning_rate': 0.01310047432090872, 'n_estimators': 848, 'max_depth': 9 -> 0.0812793649948276
# LGB : 'learning_rate': 0.0277239359067536, 'n_estimators': 809, 'max_depth': 10 -> 0.0949505112796726
# RF : 'learning_rate': 0.01846197377069944, 'n_estimators': 904, 'max_depth': 6 -> 0.07889456904387222
# GBM : 'learning_rate': 0.0035644851302199143, 'n_estimators': 943, 'max_depth': 10 -> 0.0825830066358257
# HGB : 'learning_rate': 0.06936407648125087, 'n_estimators': 503, 'max_depth': 6 -> 0.0832921275577305
# RF + HGB : 'learning_rate': 0.027136873126814776, 'n_estimators': 652, 'max_depth': 9 -> 0.0793523289383799

Best Score : 0.0793523289383799
Best trial : {'learning_rate': 0.027136873126814776, 'n_estimators': 652, 'max_depth': 9}


In [225]:
n_fold = 5
cv = KFold(n_splits = n_fold, shuffle = True, random_state = 39)

cat_val = np.zeros((X_train2.shape[0]))
cat_test = np.zeros((X_test.shape[0]))

print(cat_val.shape)
print(cat_test.shape)

for i, (i_trn, i_val) in enumerate(cv.split(X_train2, y_train2), 1):
    print(f'training model for CV #{i}')
    optuna_cat = CatBoostRegressor(
        random_state = 39,
        learning_rate = 0.01846197377069944, 
        n_estimators = 904, 
        max_depth = 6)

    optuna_cat.fit(X_train2.loc[i_trn, :], y_train2[i_trn], verbose = False, 
                   cat_features = categorical_features, early_stopping_rounds = 100)

    cat_val[i_val] = optuna_cat.predict(X_train2.loc[i_val, :])
    cat_test += optuna_cat.predict(X_test) / n_fold

(14095,)
(6041,)
training model for CV #1
training model for CV #2
training model for CV #3
training model for CV #4
training model for CV #5


In [226]:
scores = []
TP = []
FP = []
FN = []
TN = []
for threshold in range(50) :
    threshold = threshold / 50
    pred = cat_val
    pred = np.where(pred >= threshold, 1, 0)
    score = f1_score(y_train, pred)
    scores.append(score)
    TP.append(confusion_matrix(y_train, pred)[0][0])
    FN.append(confusion_matrix(y_train, pred)[0][1])
    FP.append(confusion_matrix(y_train, pred)[1][0])
    TN.append(confusion_matrix(y_train, pred)[1][1])
    

temp1 = pd.DataFrame(np.linspace(0, 0.98, 50), columns = ['threshold'])
temp2 = pd.DataFrame(scores, columns = ['score'])
temp3 = pd.DataFrame(TP, columns = ['TP'])
temp4 = pd.DataFrame(FP, columns = ['FP'])
temp5 = pd.DataFrame(FN, columns = ['FN'])
temp6 = pd.DataFrame(TN, columns = ['TN'])
scores = pd.concat([temp1, temp2, temp3, temp4, temp5, temp6], axis = 1)
scores


,threshold,score,TP,FP,FN,TN
0,0.00,0.157286,1,0,12891,1203
1,0.02,0.159368,224,2,12668,1201
2,0.04,0.175220,2077,49,10815,1154
3,0.06,0.199084,4859,182,8033,1021
4,0.08,0.226240,7423,352,5469,851
5,0.10,0.248428,9498,551,3394,652
6,0.12,0.256775,10911,734,1981,469
7,0.14,0.243053,11724,875,1168,328
8,0.16,0.224555,12150,957,742,246
9,0.18,0.209584,12395,1004,497,199


In [227]:
scores = []
for threshold in range(100) :
    threshold = threshold / 100
    pred = cat_val
    pred = np.where(pred >= threshold, 1, 0)
    score = f1_score(y_train, pred)
    scores.append(score)

temp1 = pd.DataFrame(np.linspace(0, 0.99, 100), columns = ['threshold'])
temp2 = pd.DataFrame(scores, columns = ['score'])
scores = pd.concat([temp1, temp2], axis = 1)
scores.loc[: 50, :]

,threshold,score
0,0.00,0.157286
1,0.01,0.157636
2,0.02,0.159368
3,0.03,0.166180
4,0.04,0.175220
5,0.05,0.186376
6,0.06,0.199084
7,0.07,0.211991
8,0.08,0.226240
9,0.09,0.237731


In [228]:
scores.loc[scores['score'] == scores['score'].max(), :]

,threshold,score
12,0.12,0.256775


In [232]:
answer = np.zeros(cat_test.shape[0])

for i in range(cat_test.shape[0]) :
  if cat_test[i] >= 0.147 :
    answer[i] = 1
    
answer = answer.astype('int64')
print(Counter(answer))

# CAT, LGB : 0.144 -> 523개, 0.243522
# CAT, RF : 0.147 -> 531개, 0.256775
# CAT, GBM : 0.11 -> 505개, 0.24704
# CAT, HGB : 0.154 -> 521개, 0.253708
# CAT, RF, HGB : 0.151 -> 529개, 0.254096

Counter({0: 5510, 1: 531})


In [233]:
submission_preds = answer
submission = pd.read_csv('/home/studio-lab-user/MYDATA/Construction Machine Oil/open/sample_submission.csv')
submission['Y_LABEL'] = submission_preds
submission.to_csv('/home/studio-lab-user/MYDATA/Construction Machine Oil/결과/CAT_RF.csv', index = False)